# Ideia do Projeto Slot 4 - Wallmart

Case: Você foi contratado pelo Wallmart para fazer um levantamento do faturamento das lojas nos USA e apontar qual loja seria melhor para expandir seu tamanho. É preciso analisar as vendas semanais de cada loja, calcular algumas informações importantes que serão perguntadas e ao final de tudo, indicar em qual loja deva ser investida.

# Sobre o Wallmart 
[Walmart, Inc.](https://pt.wikipedia.org/wiki/Walmart), é uma multinacional estadunidense de lojas de departamento. A companhia foi fundada por Sam Walton em 1962, incorporada em 31 de outubro de 1969 e feita capital aberto na New York Stock Exchange, em 1972. No ano de 2021, obteve um um lucro de $13.51 Bilhões.

# O Dataset

Sendo uma das principais lojas de varejo do mundo, os dados contemplam as vendas semanais de 45 lojas espalhadas pelos Estados Unidos. O Walmart realiza vários eventos promocionais de descontos ao longo do ano. Essas remarcações precedem feriados importantes, os quatro maiores de todos, que são o Super Bowl, o Dia do Trabalho, o Dia de Ação de Graças e o Natal. As semanas que incluem esses feriados têm um peso maior.

Sobre o [Dataset do Wallmart](https://www.kaggle.com/yasserh/walmart-dataset)

Os dados apresentam as vendas semanais de 05/02/10 até 01/11/12 em 45 lojas varejistas da rede Wallmart e também algumas métricas econômicas e meteriológicas por semana. A descrição destas características estão abaixo:

Variável | Descrição
--- | ---
Store | Número da Loja
Date | Semana de venda
Weekly_Sales | Venda naquela semana
Holiday_Flag | Flag se é ou não semana com feriado (1 – Holiday week 0 – Non-holiday week)
Temperature | Temperatuda do dia em ºF 
Fuel_Price | Preço do combustível na região da loja
CPI | Índice de preços ao consumidor 
Unemployment | Taxa de desemprego

Variável | Descrição
--- | ---
Holiday Events | Semanas
Super Bowl |  12-Feb-10, 11-Feb-11, 10-Feb-12
Labour Day | 10-Sep-10, 9-Sep-11, 7-Sep-12
Thanksgiving |  26-Nov-10, 25-Nov-11, 23-Nov-12
Christmas |  31-Dec-10, 30-Dec-11, 28-Dec-12

# Perguntas importantes a serem respondidas

1. Qual o preço médio de venda semanal por loja?
2. Qual o atributo que tem maior correlação (positiva ou negativa) com o número de vendas? (E que faça sentido)
3. Da loja com maior venda acumulada (soma de vendas de todo o período), quantas semanas do ano ele ultrapassou a média do período?
4. Faça um gráfico com as vendas máximas, mínimas e médias de todas as lojas ao longo do tempo. Ressalte de alguma forma, o período de feriados mencionados acima.
5. Pensando na expansão de uma das lojas, que loja você escolheria e o por que?  

## **0.0 Bibliotecas utilizadas**

In [11]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

### 0.1 Funções auxiliares

In [12]:
def personal_settings():
    
    pd.set_option('display.float_format', '{:.2f}'.format)

In [13]:
personal_settings()

## **1.0 Descrição dos dados**

### 1.1 Importação dos dados

In [14]:
# Leitura Local
#df = pd.read_csv('Walmart.csv', parse_dates=['Date'])
#print(df.shape)
#df.head()

# Leitura Cloud
URL = "https://drive.google.com/file/d/1DvzVJYA187Vv4_0b0qJnpaheqV-1784j/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+URL.split("/")[-2]
df = pd.read_csv(path, parse_dates=['Date'])

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_9420\700833101.py:9: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(path, parse_dates=['Date'])


In [15]:
# Verificar as cinco primeiras linhas
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,2010-05-02,1643690.90,0,42.31,2.57,211.10,8.11
1,1,2010-12-02,1641957.44,1,38.51,2.55,211.24,8.11
2,1,2010-02-19,1611968.17,0,39.93,2.51,211.29,8.11
3,1,2010-02-26,1409727.59,0,46.63,2.56,211.32,8.11
4,1,2010-05-03,1554806.68,0,46.50,2.62,211.35,8.11


In [16]:
# Verificar as cinco últimas linhas
df.tail()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
6430,45,2012-09-28,713173.95,0,64.88,4.00,192.01,8.68
6431,45,2012-05-10,733455.07,0,64.89,3.98,192.17,8.67
6432,45,2012-12-10,734464.36,0,54.47,4.00,192.33,8.67
6433,45,2012-10-19,718125.53,0,56.47,3.97,192.33,8.67
6434,45,2012-10-26,760281.43,0,58.85,3.88,192.31,8.67


### 1.2 Dimensão dos dados

In [17]:
print(df.shape[0], 'Linhas')
print(df.shape[1], 'Colunas')

6435 Linhas
8 Colunas


### 1.3 Tipos de dados

In [18]:
description = []

for i in df.columns:
    description.append([
        i,
        df[i].dtypes,
        df[i].nunique(),
        df[i].drop_duplicates().sample(2).values
    ])

pd.DataFrame(data=description, columns=['attributes', 'dtypes', 'nunique', 'sample'])

,attributes,dtypes,nunique,sample
0,Store,int64,45,"[33, 25]"
1,Date,datetime64[ns],143,"[2012-09-14T00:00:00.000000000, 2012-08-31T00:..."
2,Weekly_Sales,float64,6435,"[424956.3, 853073.17]"
3,Holiday_Flag,int64,2,"[0, 1]"
4,Temperature,float64,3528,"[79.75, 46.85]"
5,Fuel_Price,float64,892,"[3.013, 3.043]"
6,CPI,float64,2145,"[207.2581929, 190.3284922]"
7,Unemployment,float64,349,"[8.257, 8.442]"


### 1.4 Métricas estatísticas  inicias

In [19]:
df.describe()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,6435.00,6435.00,6435.00,6435.00,6435.00,6435.00,6435.00
mean,23.00,1046964.88,0.07,60.66,3.36,171.58,8.00
std,12.99,564366.62,0.26,18.44,0.46,39.36,1.88
min,1.00,209986.25,0.00,-2.06,2.47,126.06,3.88
25%,12.00,553350.10,0.00,47.46,2.93,131.74,6.89
50%,23.00,960746.04,0.00,62.67,3.44,182.62,7.87
75%,34.00,1420158.66,0.00,74.94,3.73,212.74,8.62
max,45.00,3818686.45,1.00,100.14,4.47,227.23,14.31


## **2.0 Limpeza de dados (Parte 01)**

### 2.1 Identificando valores ausentes

In [20]:
description = []
for i in df.columns:
    description.append([
        i,
        df[i].isna().sum(),
        round((df[i].isna().sum()/len(df))*100)
    ])

pd.DataFrame(data=description, columns=['attributes','null', '%null'])

#se a soma de todos for 0 é um excelente resultado e a base não precisa ser tratada

,attributes,null,%null
0,Store,0,0
1,Date,0,0
2,Weekly_Sales,0,0
3,Holiday_Flag,0,0
4,Temperature,0,0
5,Fuel_Price,0,0
6,CPI,0,0
7,Unemployment,0,0


### 2.2 Identificando duplicadas

In [21]:
# verificar duplicados
df.duplicated().any()

False

In [22]:
# verificar missing values
df.isna().sum().any()

False